In [ ]:
import cv2

In [ ]:
import seed_segmenter

vidcap = cv2.VideoCapture('raw/kaura_1.avi', None)

nth_frame = 30
images = []
frame_counter = 0
frame2_counter = 0

while True:
    success, image = vidcap.read()
    if not success: break

    frame_counter += 1
    if frame_counter == nth_frame:
        frame_counter = 0
        was = len(images) >= 55
        images += seed_segmenter.separate_seeds(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if len(images) == 55 and not was:
            print(frame2_counter)
        frame2_counter += 1

In [ ]:
from pathlib import Path
from PIL import Image
import shutil

data_dir = Path('extracted_data')
shutil.rmtree(data_dir)
data_dir.mkdir()

for i, image in enumerate(images):
    Image.fromarray(image).save(data_dir / f'seed_{i}.png', )

In [ ]:
import threading
import skimage
import shutil
from pathlib import Path
from PIL import Image
from seed_segmenter import separate_seeds
import numpy as np
import concurrent.futures

# From the actual images

in_images = Path('raw_image/data')
out_images = Path('extracted_data_from_images')
shutil.rmtree(out_images, True)

for category in in_images.iterdir():
    out_dir = out_images / category.relative_to(in_images)
    out_dir.mkdir(parents=True)
    i = 1
    i_lock = threading.Lock()

    def handle_image(imfile):
        global i
        img = np.array(Image.open(imfile))
        separated = separate_seeds(
            img,
            crop_left_right=(40, 1340),
            threshold=0.01,
            close_kernel=skimage.morphology.disk(3),
            open_kernel=skimage.morphology.disk(5),
            white=(1.0, 0.9, 0.8),
            median_filter=3,
            minimum_area=2000)
        for s in separated:
            with i_lock:
                this_i = i
                i += 1
            print(f'{this_i}: {imfile}')
            Image.fromarray(s).save(out_dir / f'seed_{this_i}.png', optimize=True)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for imfile in category.iterdir():
            executor.submit(handle_image, imfile)
